In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sc_employment = pd.read_csv(
    "./data/processed_employment/Santa_Clara_County,_California.csv"
)
sc_real_estate = pd.read_csv("./data/processed_zillow/Santa_Clara_County.csv")

data = sc_real_estate
data = pd.concat([data, sc_employment], axis=1)
data = data.dropna(subset=["Year-Month"])
data["county"] = "Santa Clara"
data["constant"] = 1
data["Date"] = pd.to_datetime(data["Date"])
data["Year"] = data["Date"].dt.year
data["Quarter"] = data["Date"].dt.quarter
data["Year_Quarter"] = data["Year"].astype(str) + "Q" + data["Quarter"].astype(str)
year_quarter_dummies = pd.get_dummies(
    data["Year_Quarter"], prefix="YQ", drop_first=True
)
data = pd.concat([data, year_quarter_dummies], axis=1)

data = data.set_index(["county", "Date"])

mod = PanelOLS(data.Price, data[["Employment"] + ["constant"]])
# mod = PanelOLS(
#     data.Price,
#     data[["Employment"] + year_quarter_dummies.columns.tolist() + ["constant"]],
# )
res = mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Price   R-squared:                        0.9986
Estimator:                   PanelOLS   R-squared (Between):              0.0000
No. Observations:                 159   R-squared (Within):               0.9986
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.9986
Time:                        13:55:57   Log-likelihood                   -1721.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1424.3
Entities:                           1   P-value                           0.0000
Avg Obs:                       159.00   Distribution:                  F(53,105)
Min Obs:                       159.00                                           
Max Obs:                       159.00   F-statistic (robust):             1424.3
                            

In [ ]:
# Attempt to format table

with open("regression_results.tex", "w") as f:
    f.write(res.summary.as_latex())



In [5]:
from stargazer.stargazer import Stargazer

# Create Stargazer object
stargazer = Stargazer([res])

# Export LaTeX table
with open("regression_results.tex", "w") as f:
    f.write(stargazer.render_latex())


In [33]:
from statsmodels.iolib.summary2 import summary_col

# Choose specific parameters to display, e.g., the first five
selected_params = res.params.head(5).index
print(selected_params)

# Create a new DataFrame with just the selected parameters
filtered_results = res.params[selected_params].to_frame("coef")
filtered_results["std err"] = res.std_errors[selected_params]
filtered_results["t"] = res.tstats[selected_params]
filtered_results["P>|t|"] = res.pvalues[selected_params]
filtered_results["[0.025"] = res.conf_int().loc[selected_params, 0]
filtered_results["0.975]"] = res.conf_int().loc[selected_params, 1]

# Display the filtered results in a summary table format
print(summary_col([filtered_results], stars=True, float_format="%0.4f"))

Index(['Employment', 'YQ_2011Q2', 'YQ_2011Q3', 'YQ_2011Q4', 'YQ_2012Q1'], dtype='object')


KeyError: 0